# Инсталляция библиотек

In [ ]:
pip install datasets evaluate

# Очистка данных

In [ ]:
import pandas as pd
import glob
import chardet
import os

# Ищем все файлы сезонов
season_files = sorted(glob.glob("/content/season*.csv"))

df_list = []

for file in season_files:
    # Определяем кодировку перед чтением
    with open(file, "rb") as f:
        result = chardet.detect(f.read(100000))  # Анализ первых 100кб
        encoding = result["encoding"]

    print(f"Читаем файл {file} с кодировкой {encoding}")

    try:
        # Читаем CSV-файл с определенной кодировкой
        df = pd.read_csv(file, encoding=encoding, on_bad_lines="skip")
        df["season"] = os.path.basename(file).split(".")[0]  # Добавляем сезон
        df_list.append(df)
    except Exception as e:
        print(f"Ошибка при чтении {file}: {e}")

# Объединяем все файлы в один DataFrame
if df_list:
    full_dataset = pd.concat(df_list, ignore_index=True)
    print(f"\nФинальный размер объединенного датасета: {full_dataset.shape}")

    # Приведение имени персонажа к единому формату
    full_dataset["name"] = full_dataset["name"].astype(str).str.strip().str.lower()

    # Добавление колонок контекста
    full_dataset["context_1"] = None
    full_dataset["context_1_character"] = None
    full_dataset["context_2"] = None
    full_dataset["context_2_character"] = None
    full_dataset["context_3"] = None
    full_dataset["context_3_character"] = None

    # Заполнение контекста только для строк, где name = 'house'
    for i in range(3, len(full_dataset)):
        if full_dataset.loc[i, "name"] == "house":
            full_dataset.at[i, "context_1"] = full_dataset.loc[i - 1, "line"]
            full_dataset.at[i, "context_1_character"] = full_dataset.loc[i - 1, "name"]

            full_dataset.at[i, "context_2"] = full_dataset.loc[i - 2, "line"]
            full_dataset.at[i, "context_2_character"] = full_dataset.loc[i - 2, "name"]

            full_dataset.at[i, "context_3"] = full_dataset.loc[i - 3, "line"]
            full_dataset.at[i, "context_3_character"] = full_dataset.loc[i - 3, "name"]

    # Сохранение полного датасета с контекстом
    full_output_file = "/content/house_full_dataset_with_context.csv"
    full_dataset.to_csv(full_output_file, index=False, encoding="utf-8")
    print(f"\nОбновленный датасет сохранен: {full_output_file}")

    # Фильтрация реплик Доктора Хауса с контекстом
    house_lines = full_dataset[full_dataset["name"] == "house"]

    # Сохранение реплик Хауса
    house_output_file = "/content/house_lines_with_context.csv"
    house_lines.to_csv(house_output_file, index=False, encoding="utf-8")

In [ ]:
import os
import shutil
import re
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util, models, losses, evaluation, InputExample
from transformers import (
    AutoConfig, AutoModel, AutoTokenizer,
    AutoModelForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
)
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import evaluate
from google.colab import files

# Определение устройства для работы с моделью (CPU или GPU)
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Пути к данным
house_path = "/content/data/house_lines_with_context.csv"
antagonists_path = "/content/data/all_series_cleaned.csv"

# Загрузка данных
house_df = pd.read_csv(house_path)
antagonists_df = pd.read_csv(antagonists_path)

# Вывод первых строк для проверки
print("Первые строки датасета Доктора Хауса:")
print(house_df.head())

print("Первые строки датасета антагонистов:")
print(antagonists_df.head())

Первые строки датасета Доктора Хауса:
    name                                               line   season  \
0  house   See that? They all assume I'm a Patient becau...  season1   
1  house           I don't want them to think I'm a doctor.  season1   
2  house                   People don't want a sick doctor.  season1   
3  house         The one who can't talk, I liked that part.  season1   
4  house   And your cousin doesn't like the diagnosis. I...  season1   

                                           context_1 context_1_character  \
0   29 year old female, first seizure one month a...              wilson   
1        So put on a white coat like the rest of us.              wilson   
2   You see where the administration might have a...              wilson   
3   Fair enough. I don't like healthy Patients. T...              wilson   
4                                   She's my cousin.              wilson   

                                           context_2 context_2_character

In [ ]:
import pandas as pd
import re

# --- ФУНКЦИИ ОЧИСТКИ ---
def clean_text(text):
    """Очистка текста от проблемных символов, лишних пробелов и мусора."""
    if not isinstance(text, str) or len(text) == 0:
        return ""

    text = text.replace("�", "")  # Удаление неизвестных символов
    text = re.sub(r"\s+", " ", text).strip()  # Убираем лишние пробелы
    text = re.sub(r"[^\x00-\x7F]+", "", text)  # Удаляем любые не-ASCII символы (если есть)

    return text

def additional_cleaning(text):
    """Дополнительная очистка с учетом разрешенных символов."""
    if not isinstance(text, str):
        return text

    text = re.sub(r"[^\w\s.,!?'$/%-]", "", text)  # Оставляем только буквы, цифры и базовые знаки препинания
    text = re.sub(r"\s+", " ", text).strip()  # Удаление лишних пробелов

    return text

def apply_text_cleaning(df, text_columns):
    """Очистка всех указанных текстовых колонок."""
    for col in text_columns:
        df[col] = df[col].astype(str).apply(clean_text).apply(additional_cleaning)
    return df

# --- ОЧИСТКА ДАННЫХ ---
# Определяем нужные колонки
text_columns_house = ["line", "context_1", "context_2", "context_3"]
text_columns_antagonists = ["line"]

# Применяем очистку к датасетам
house_df = apply_text_cleaning(house_df, text_columns_house)
antagonists_df = apply_text_cleaning(antagonists_df, text_columns_antagonists)

# Выводим итоговую статистику
print(f"\nКоличество реплик после очистки: {len(house_df)} (Доктор Хаус), {len(antagonists_df)} (Антагонисты).")



Количество реплик после очистки: 21826 (Доктор Хаус), 167345 (Антагонисты).


In [ ]:
def is_question(text):
    """Определяет, содержит ли реплика вопрос"""
    if not isinstance(text, str):
        return False

    # Проверка, заканчивается ли текст на вопросительный знак
    if text.strip().endswith("?"):
        return True

    # Поиск вопросительных слов в тексте
    question_words = {"what", "why", "how", "who", "when", "where", "which",
                      "что", "почему", "как", "кто", "когда", "где", "какой", "зачем"}

    words = set(text.lower().split())
    if question_words.intersection(words):
        return True

    # Проверка наличия вопросительного знака внутри реплики
    if "?" in text:
        return True

    # Проверка восклицательных предложений, содержащих вопросительные слова
    if text.endswith("!") and question_words.intersection(words):
        return True

    return False

def analyze_questions(df, context_columns):
    """Подсчитывает количество вопросов в указанных колонках"""
    question_stats = {col: df[col].apply(is_question).sum() for col in context_columns}
    return question_stats

# Определение колонок для анализа
context_columns = ["line", "context_1", "context_2", "context_3"]

# Анализ вопросов в контексте
question_stats = analyze_questions(house_df, context_columns)

# Вывод статистики по вопросам
print("\nКоличество вопросов в предыдущих репликах:")
for col, count in question_stats.items():
    total = len(house_df)
    print(f"{col}: {count} ({count / total * 100:.2f}%)")

# Выборка примеров реплик с вопросами
print("\nПримеры вопросов в предыдущих репликах:")
for col in context_columns:
    sample_questions = house_df[house_df[col].apply(is_question)][col].dropna().sample(5, random_state=42).tolist()
    print(f"\n{col}:")
    for q in sample_questions:
        print(f"- {q}")



Количество вопросов в предыдущих репликах:
line: 7983 (36.58%)
context_1: 7288 (33.39%)
context_2: 7595 (34.80%)
context_3: 7313 (33.51%)

Примеры вопросов в предыдущих репликах:

line:
- And if she whacks herself, what happens to your job?
- See? There are those eyes again. He moves toward the desk and Adams grabs Nick's folder and holds it away from House. How about a viral syndrome? Or MRSA? No, that wouldn't chew up the bone that fast. But reaction to meds could. holds out his hand for the chart May I?
- I am so ashamed. to Wilson She really that good looking?
- For you, or me?
- Turns back to the Patient. Any fEver?

context_1:
- What are what are you doing? What are you doing?
- Do you know why people beliEve in God?
- She was screaming she was being sucked into a black hole, so, we should look for a tumor in her anus?
- Where are we going?
- interjects loudly PRobably? You think there's a possibility that the alternative is true?

context_2:
- looking at his yo-yo I can't remem

In [ ]:
# Функция для вычисления статистики по длине текста
def text_length_stats(df, text_columns):
    stats = {}
    for col in text_columns:
        lengths = df[col].dropna().astype(str).apply(len)  # Длина строк
        stats[col] = {
            "min": lengths.min(),
            "max": lengths.max(),
            "mean": lengths.mean(),
            "median": lengths.median(),
            "std": lengths.std()
        }
    return pd.DataFrame(stats)

# Определяем колонки для анализа
text_columns = ["line", "context_1", "context_2", "context_3"]

# Вычисляем статистику
stats_df = text_length_stats(house_df, text_columns)

stats_df

,line,context_1,context_2,context_3
min,2.000000,0.000000,0.000000,0.000000
max,1293.000000,1293.000000,1079.000000,1293.000000
mean,84.425822,62.191286,77.118483,64.355906
median,61.000000,45.000000,56.000000,46.000000
std,80.980381,59.355878,74.420721,61.907518


In [ ]:
import re

def truncate_long_text(text, max_words=30):
    """Обрезает текст после первого найденного знака препинания (. ? !) или по количеству слов."""
    if not isinstance(text, str):
        return text

    words = text.split()

    # Если реплика короче max_words, оставляем как есть
    if len(words) <= max_words:
        return text

    # Ищем ближайший знак препинания (. ? !)
    match = re.search(r"([.!?])\s", text)

    if match:
        cutoff_index = match.end()  # Обрезаем после знака
        return text[:cutoff_index].strip()

    # Если знаков нет — оставляем первые max_words слов
    return " ".join(words[:max_words]).strip()

# Применяем к колонке 'line' (ответы Хауса)
house_df["line"] = house_df["line"].apply(lambda x: truncate_long_text(x, max_words=30))

print("✅ Длинные реплики Хауса обрезаны по знакам препинания или количеству слов (30 слов макс.).")


✅ Длинные реплики Хауса обрезаны по знакам препинания или количеству слов (30 слов макс.).


In [ ]:
# Функция для подсчёта слов в реплике
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# Подсчитываем количество реплик, где больше 50 слов
long_replies = house_df[house_df["line"].apply(count_words) > 50]

# Количество таких реплик
long_replies_count = len(long_replies)

print(f"📊 Количество реплик, где >50 слов: {long_replies_count}\n")

# Выводим 10 самых длинных реплик
longest_replies = long_replies.sort_values(by="line", key=lambda x: x.str.len(), ascending=False).head(10)

longest_replies

📊 Количество реплик, где >50 слов: 0



,name,line,season,context_1,context_1_character,context_2,context_2_character,context_3,context_3_character


In [ ]:
# Функция для подсчёта слов в тексте
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# Создаём колонку anchor: копируем context_1, если он больше 10 слов
house_df["anchor"] = house_df["context_1"].apply(lambda x: x if count_words(x) > 10 else "")

# Подсчитываем количество строк с длинными anchor (> 50 слов)
long_anchor_count = (house_df["anchor"].apply(count_words) > 50).sum()

# Вывод статистики
print(f"Количество строк, где 'anchor' не пустой: {house_df['anchor'].str.len().gt(0).sum()}")
print(f"Количество слишком длинных 'anchor' (>50 слов): {long_anchor_count}")

Количество строк, где 'anchor' не пустой: 8638
Количество слишком длинных 'anchor' (>50 слов): 252


In [ ]:
import re

def truncate_anchor(text, max_words=50):
    """Обрезает anchor, оставляя 50 слов с конца + до ближайшего знака препинания (.?!)."""
    if not isinstance(text, str):
        return text

    words = text.split()

    # Если текст короче max_words, оставляем как есть
    if len(words) <= max_words:
        return text

    # Оставляем последние max_words слов
    truncated_text = " ".join(words[-max_words:])

    # Ищем ближайший знак препинания (вперёд от обрезанного конца)
    match = re.search(r"[.!?]", truncated_text)

    if match:
        cutoff_index = match.end()  # Обрезаем после знака
        return truncated_text[:cutoff_index].strip()

    return truncated_text.strip()  # Если знаков нет, оставляем как есть

# Применяем обрезку только к слишком длинным anchor
house_df["anchor"] = house_df["anchor"].apply(lambda x: truncate_anchor(x) if count_words(x) > 50 else x)

# Проверяем, сколько длинных anchor осталось
long_anchor_count_after = (house_df["anchor"].apply(count_words) > 70).sum()

print(f"Количество слишком длинных 'anchor' (>70 слов) после обрезки: {long_anchor_count_after}")

Количество слишком длинных 'anchor' (>70 слов) после обрезки: 0


In [ ]:
# Фильтруем строки, где anchor пустой
empty_anchor_df = house_df[house_df["anchor"].str.len() == 0]

# Подсчитываем количество пустых anchor
empty_anchor_count = len(empty_anchor_df)

# Статистика по длине context_1 и context_2 в этих строках
context_stats = {
    "context_1": {
        "min": empty_anchor_df["context_1"].str.len().min(),
        "max": empty_anchor_df["context_1"].str.len().max(),
        "mean": empty_anchor_df["context_1"].str.len().mean(),
        "median": empty_anchor_df["context_1"].str.len().median(),
        "std": empty_anchor_df["context_1"].str.len().std(),
    },
    "context_2": {
        "min": empty_anchor_df["context_2"].str.len().min(),
        "max": empty_anchor_df["context_2"].str.len().max(),
        "mean": empty_anchor_df["context_2"].str.len().mean(),
        "median": empty_anchor_df["context_2"].str.len().median(),
        "std": empty_anchor_df["context_2"].str.len().std(),
    }
}

# Вывод статистики
print(f"Количество строк с пустым 'anchor': {empty_anchor_count}\n")
print("Статистика по длине контекста:")
for col, stats in context_stats.items():
    print(f"\n{col}:")
    for key, value in stats.items():
        print(f"  {key}: {value}")


Количество строк с пустым 'anchor': 13188

Статистика по длине контекста:

context_1:
  min: 0
  max: 93
  mean: 29.56756141947225
  median: 28.0
  std: 14.946127577574105

context_2:
  min: 2
  max: 1079
  mean: 76.40498938428875
  median: 54.0
  std: 75.7249535767355


In [ ]:
# Функция для подсчёта слов
def count_words(text):
    return len(text.split()) if isinstance(text, str) else 0

# Фильтруем строки для каждого количества слов
for i in range(10, 4, -1):  # От 10 до 5 слов
    sample = house_df[house_df["context_1"].apply(count_words) == i].head(5)
    print(f"\nПримеры 'context_1' с {i} словами:")
    print(sample["context_1"].tolist())


Примеры 'context_1' с 10 словами:
['I was expecting you in my office 20 minutes ago.', "You're 6 years behind on your obligation to this clinic.", 'I made sure your first case was an interesting one.', 'heavy sigh Ya know, I think there just might be.', 'Yes, yes, her name is Rebecca. I call her Rachel.']

Примеры 'context_1' с 9 словами:
['Psitticosis can lead to nerve pRoblems and neurological complications.', "Whoa, oh, I can't just break into someone's House.", 'Not that I could tell from her underwear drawer.', 'I just want to die with a little dignity.', 'I worked very hard to get where I am.']

Примеры 'context_1' с 8 словами:
['You think we have nothing to talk about?', "You also can't make long distance phone calls.", 'I was thinking it also might be fibromyalgia.', 'I solved the case, my work is done.', 'You hired me to get into my pants?!']

Примеры 'context_1' с 7 словами:
["And she's not responding to radiation treatment.", "Isn't treating Patients why we became doctors?"

In [ ]:
# Фильтруем строки для каждого количества слов
for i in range(4, 0, -1):  # От 4 до 1 слов
    sample = house_df[house_df["context_1"].apply(count_words) == i].head(5)
    print(f"\nПримеры 'context_1' с {i} словами:")
    print(sample["context_1"].tolist())



Примеры 'context_1' с 4 словами:
['I sign your paychecks.', 'I thought Everybody lied?', "You couldn't have knocked?", "And if we're wrong?", "She's a kindergarten teacher!"]

Примеры 'context_1' с 3 словами:
["She's my cousin.", 'No family history.', "It's a lesion.", 'Patient is orange.', 'No, the fruit.']

Примеры 'context_1' с 2 словами:
['No, but!', 'Mad cow?', "Wernickie's encephalopathy?", 'To what?', "It's cold."]

Примеры 'context_1' с 1 словами:
['What?!', '20', 'Right.', 'Nothing.', 'Hey!listen!']


In [ ]:
# Обновляем логику заполнения anchor: теперь переносим context_1, если в нем 3+ слова
house_df["anchor"] = house_df["context_1"].apply(lambda x: x if count_words(x) >= 3 else "")

# Подсчитываем количество строк, где anchor не пустой
filled_anchor_count = house_df["anchor"].str.len().gt(0).sum()

# Подсчитываем количество строк, где anchor остался пустым
empty_anchor_df = house_df[house_df["anchor"].str.len() == 0]
empty_anchor_count = len(empty_anchor_df)

# Подсчитываем сколько строк с пустым anchor имеют 0, 1 или 2 слова в context_1
empty_anchor_stats = {
    words: (empty_anchor_df["context_1"].apply(count_words) == words).sum()
    for words in [0, 1, 2]
}

# Вывод статистики
print(f"Количество строк, где 'anchor' не пустой: {filled_anchor_count}")
print(f"Количество строк с пустым 'anchor': {empty_anchor_count}")
print("\nРаспределение среди пустых 'anchor' по количеству слов в 'context_1':")
for key, value in empty_anchor_stats.items():
    print(f"  {key} слов(а): {value}")

Количество строк, где 'anchor' не пустой: 19842
Количество строк с пустым 'anchor': 1984

Распределение среди пустых 'anchor' по количеству слов в 'context_1':
  0 слов(а): 3
  1 слов(а): 870
  2 слов(а): 1111


In [ ]:
# Удаляем строки, где context_1 пустой (0 слов)
house_df = house_df[house_df["context_1"].str.len() > 0].reset_index(drop=True)

# Фильтруем строки, где context_1 содержит 1 или 2 слова
short_context_df = house_df[house_df["context_1"].apply(count_words).isin([1, 2])]

# Группируем и считаем самые частые context_1
top_short_contexts = (
    short_context_df.groupby("context_1").size().reset_index(name="count").sort_values(by="count", ascending=False).head(10)
)

# Выводим примеры (контекст + ответ Хауса)
print("\n10 самых часто встречающихся коротких context_1 (1-2 слова) с ответами Хауса:\n")
for _, row in top_short_contexts.iterrows():
    context = row["context_1"]
    count = row["count"]
    replies = short_context_df[short_context_df["context_1"] == context]["line"].unique()[:3]  # Берём 3 уникальных ответа Хауса
    print(f"'{context}' (встречается {count} раз)")
    for reply in replies:
        print(f"  - {reply}")
    print()



10 самых часто встречающихся коротких context_1 (1-2 слова) с ответами Хауса:

'No.' (встречается 108 раз)
  - But you are damaged, aren't you?
  - Well, then, Dave!
  - Here on work?

'What?' (встречается 54 раз)
  - Don't move. Did I bore you in there?
  - Husband described her as being unusually irritable recently.
  - Nice respectful Asian kid does the laundry.

'Why?' (встречается 40 раз)
  - 'Cause we need all the information we can get. The healthy kid can be our control group.
  - I find your interest interesting.
  - Cause the husband's not sick.

'Yeah.' (встречается 36 раз)
  - How'd you talk him down?
  - Cheers. Dry swallows the pill and Funsten eats his ewwwww Limps back
  - Now, why would you drive 70 miles to get treatment for a condition that a 9 year old could diagnose? It's the free-flowing pus that's the tip-off.

'Yes.' (встречается 29 раз)
  - Too bad, you're right.
  - It's pRobably an allergic reaction.
  - After you got so freaked about the sick babies a while

In [ ]:
import re

# Функция для извлечения последнего предложения из текста
def get_last_sentence(text):
    if not isinstance(text, str) or len(text) == 0:
        return ""
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Разделяем текст на предложения
    return sentences[-1] if sentences else text  # Берём последнее предложение

# Обновляем anchor: если он пустой, добавляем последнее предложение из context_2 + context_1
house_df["anchor"] = house_df.apply(
    lambda row: row["anchor"] if row["anchor"] else f"{get_last_sentence(row['context_2'])} {row['context_1']}".strip(), axis=1
)

# Подсчитываем количество оставшихся пустых anchor
empty_anchor_count_after = house_df["anchor"].str.len().eq(0).sum()

# Выводим статистику
print(f"Количество оставшихся пустых 'anchor' после заполнения: {empty_anchor_count_after}")


Количество оставшихся пустых 'anchor' после заполнения: 0


In [ ]:
# Подсчёт слов в anchor
house_df["anchor_word_count"] = house_df["anchor"].apply(count_words)

# Подсчитываем количество коротких anchor (<5 слов)
short_anchor_count = (house_df["anchor_word_count"] < 5).sum()

# Выводим статистику по длине anchor
anchor_stats = {
    "min": house_df["anchor_word_count"].min(),
    "max": house_df["anchor_word_count"].max(),
    "mean": house_df["anchor_word_count"].mean(),
    "median": house_df["anchor_word_count"].median(),
    "std": house_df["anchor_word_count"].std(),
}

# Вывод результатов
print(f"Количество anchor с менее чем 5 словами: {short_anchor_count}\n")
print("Статистика по длине anchor:")
for key, value in anchor_stats.items():
    print(f"  {key}: {value}")

# Вывести 10 самых коротких anchor
shortest_anchors = house_df[house_df["anchor_word_count"] < 5][["anchor", "line"]].head(10)
print("\n10 самых коротких anchor:\n")
for _, row in shortest_anchors.iterrows():
    print(f"Anchor: {row['anchor']}")
    print(f"House: {row['line']}\n")


Количество anchor с менее чем 5 словами: 3646

Статистика по длине anchor:
  min: 2
  max: 238
  mean: 12.101865004811437
  median: 9.0
  std: 10.661105791930531

10 самых коротких anchor:

Anchor: She's my cousin.
House: And your cousin doesn't like the diagnosis. I wouldn't either. Brain tumor, she's gonna die, boring.

Anchor: No family history.
House: I thought your uncle died of cancer.

Anchor: It's a lesion.
House: And the big green thing in the middle of the bigger blue thing on a map is an island. I was hoping for something a bit more creative.

Anchor: Creutzfeld-Jakob disease. Mad cow?
House: Mad zebra.

Anchor: Mad zebra. Wernickie's encephalopathy?
House: No, blood thiamine lEvel was normal.

Anchor: I sign your paychecks.
House: I have tenure. Are you going to grab my cane now, stop me from leaving?

Anchor: I thought Everybody lied?
House: Truth begins in lies. Think about it.

Anchor: Patient is orange.
House: The color?

Anchor: No, the fruit.
House: You mean yellow it

In [ ]:
# Проверяем количество полных дубликатов (anchor + line)
duplicate_count = house_df.duplicated(subset=["anchor", "line"]).sum()

# Выбираем и выводим 10 примеров дубликатов
duplicates = house_df[house_df.duplicated(subset=["anchor", "line"], keep=False)]

# Вывод статистики
print(f"Количество дубликатов (одинаковые anchor + line): {duplicate_count}")

# Вывод 10 примеров
print("\n10 примеров дубликатов:\n")
print(duplicates[["anchor", "line"]].head(10))

Количество дубликатов (одинаковые anchor + line): 1

10 примеров дубликатов:

              anchor    line
19169  spoken spoken  spoken
19170  spoken spoken  spoken


In [ ]:
# Удаляем дубликаты по anchor + line, оставляя первую встречающуюся строку
house_df = house_df.drop_duplicates(subset=["anchor", "line"]).reset_index(drop=True)

print("Дубликаты удалены.")

Дубликаты удалены.


In [ ]:
def clean_text_general(text):
    """Удаляет лишние пробелы и исправляет форматирование во всех текстовых колонках."""
    if not isinstance(text, str):
        return ""
    return text.strip()

# Применяем очистку ко всем текстовым колонкам
text_columns = ["anchor", "context_1", "context_2", "context_3", "line"]
for col in text_columns:
    house_df[col] = house_df[col].apply(clean_text_general)

# Проверяем результат на случайных строках
print("\nПримеры очищенных данных:")
print(house_df.sample(5, random_state=42)[text_columns])



Примеры очищенных данных:
                                                  anchor  \
12608           You'd jeopardize a Patient because of my   
20683                                       Let me down.   
12465                                     ... for these.   
1807   So, if you ignore ethics to save one person it...   
20345  No, we already tested his liver enzymes. They'...   

                                               context_1  \
12608           You'd jeopardize a Patient because of my   
20683                                       Let me down.   
12465                                     ... for these.   
1807   So, if you ignore ethics to save one person it...   
20345  No, we already tested his liver enzymes. They'...   

                                               context_2  \
12608  Your conscience bleeds more freely than my hea...   
20683                                          Admit it.   
12465                           I assume you are here...   
1807   What

In [ ]:
total_replicas = len(antagonists_df)
unique_replicas = antagonists_df["line"].nunique()
duplicates_count = total_replicas - unique_replicas

print(f"Всего реплик: {total_replicas}")
print(f"Уникальных реплик: {unique_replicas}")
print(f"Количество повторяющихся реплик: {duplicates_count}")

# Удаление повторяющихся реплик
antagonists_df = antagonists_df.drop_duplicates(subset=["line"]).reset_index(drop=True)

# Проверка
print(f"✅ Размер антагонистов после удаления повторов: {antagonists_df.shape[0]}")



Всего реплик: 167345
Уникальных реплик: 162830
Количество повторяющихся реплик: 4515
✅ Размер антагонистов после удаления повторов: 162830


In [ ]:
# Указываем путь для сохранения
house_cleaned_path = "/content/data/house_extended_cleaned.csv"

antagonists_сleaned_path = "/content/data/antagonists_extended_cleaned.csv"

# Сохранение датасетов
house_df.to_csv(house_cleaned_path, index=False, encoding="utf-8")
antagonists_df.to_csv(antagonists_сleaned_path, index=False, encoding="utf-8")

print(f"Очищенный датасет доктора Хауса сохранён в: {house_cleaned_path}")
print(f"Очищенный датасет антигероев сохранён в: {antagonists_сleaned_path}")

Очищенный датасет доктора Хауса сохранён в: /content/data/house_extended_cleaned.csv
Очищенный датасет антигероев сохранён в: /content/data/antagonists_extended_cleaned.csv


In [ ]:
antagonists_df

,character,line,source
0,WAYMAR ROYCE,What dyou expect? Theyre savages. One lot stea...,Game_of_Thrones
1,WILL,Ive never seen wildlings do a thing like this....,Game_of_Thrones
2,WAYMAR ROYCE,How close did you get?,Game_of_Thrones
3,WILL,Close as any man would.,Game_of_Thrones
4,GARED,We should head back to the wall.,Game_of_Thrones
...,...,...,...
162825,LANDO,I promised to return his ship without a scratc...,Star_Wars
162826,THREEPIO,They did it!,Star_Wars
162827,"""40"" ""LUKE"" ""Greetings, Exalted One. Allow me ...",these two droids.,Star_Wars
162828,"""146"" ""THREEPIO"" ""Victims of the almighty Sarlacc",His Excellency hopes that you will die honorab...,Star_Wars


In [ ]:
house_df

,name,response,season,context_1,context_1_character,context_2,context_2_character,context_3,context_3_character,anchor,anchor_word_count,neg_response
0,house,See that? They all assume I'm a Patient becaus...,season1,"29 year old female, first seizure one month ag...",wilson,"We know that word, the",boy,the,sidney,"29 year old female, first seizure one month ag...",23,This has been a really tough time.
1,house,I don't want them to think I'm a doctor.,season1,So put on a white coat like the rest of us.,wilson,See that? They all assume I'm a Patient becaus...,house,"29 year old female, first seizure one month ag...",wilson,So put on a white coat like the rest of us.,11,Want to come to dinner with us?
2,house,People don't want a sick doctor.,season1,You see where the administration might have a ...,wilson,I don't want them to think I'm a doctor.,house,So put on a white coat like the rest of us.,wilson,You see where the administration might have a ...,12,"Yeah. Oh, OOOH, yeah, you know, did you notice..."
3,house,"The one who can't talk, I liked that part.",season1,Fair enough. I don't like healthy Patients. Th...,wilson,People don't want a sick doctor.,house,You see where the administration might have a ...,wilson,Fair enough. I don't like healthy Patients. Th...,12,"Yes, but I dont think life experience with these."
4,house,And your cousin doesn't like the diagnosis. I ...,season1,She's my cousin.,wilson,"The one who can't talk, I liked that part.",house,Fair enough. I don't like healthy Patients. Th...,wilson,She's my cousin.,3,"Yknow, thanks for trying to cheer me up, but I..."
...,...,...,...,...,...,...,...,...,...,...,...,...
21817,house,Hi.,season8,"Phone. A million times he needed me, and the o...",wilson,House was an ass. That gets Everyone's attenti...,wilson,reading his notes He was my friend. The thing ...,wilson,"Phone. A million times he needed me, and the o...",81,How is this on me?
21818,house,I got out of the back of the building.,season8,How?,wilson,Hi.,house,"Phone. A million times he needed me, and the o...",wilson,Hi. How?,2,Nein! Im greek! My name is Mikanos.
21819,house,Just switched the dental records.,season8,The body,wilson,I got out of the back of the building.,house,How?,wilson,I got out of the back of the building. The body,11,Stop that! Stop that!
21820,house,"I'm dead, Wilson. How do you want to spend you...",season8,You're destroying your entire life. You can't ...,wilson,Just switched the dental records.,house,The body,wilson,You're destroying your entire life. You can't ...,24,I thought the Nights Watch might make a man of...


In [ ]:
# Сохранение нужных колонок в CSV
house_df[['anchor', 'response']].to_csv('questions_answers.csv', index=False)